In [1]:
import os
import imageio
import numpy as np

In [10]:
path = '../ColabNotebooks/dataset'

In [3]:
def extractFrames(file_name, frame_step):
  video = imageio.get_reader(file_name, 'ffmpeg')
  return np.array([frame for i, frame in enumerate(video) if i % frame_step == 0])

In [4]:
def preProcessing(frames):
  video_r_channel = np.zeros(frames.shape[0:3])
  video_g_channel = np.zeros(frames.shape[0:3])
  video_b_channel = np.zeros(frames.shape[0:3])

  for i, frame in enumerate(frames):
    for j, row in enumerate(frame):
      for k, pixel in enumerate(row):
        video_b_channel[i][j][k] = pixel[0]
        video_g_channel[i][j][k] = pixel[1]
        video_r_channel[i][j][k] = pixel[2]
        
  return video_r_channel, video_g_channel, video_b_channel

In [5]:
# Returns correlation value
def superimpose(frame: np.ndarray, kernel: np.ndarray):
  n = kernel.shape[0]
  offset = n // 2

  superimposed_matrix = frame * kernel
  return sum(superimposed_matrix.flatten()) - superimposed_matrix[offset][offset]

In [6]:
def isValid(index, length, skip):
  return index >= skip and index < length - skip

In [7]:
# Channel is a 3D matrix: (number of frames, height, width)
# each element represents the pixel value of the given channel

def convolve(channel, kernel):
  n = len(kernel) # Kernel is of dimension n x n
  skip = n // 2 # No. of rows and cols to be skipped

  num_frames, height, width = channel.shape

  # res_frames = np.zeros((len(frames), height - 2 * skip, width - 2 * skip))
  res_channel = np.zeros(channel.shape)

  for i, frame in enumerate(channel):
    print(f'Frame: {i}')

    for j, row in enumerate(frame):
      if not isValid(j, height, skip):
        continue

      for k, pixel in enumerate(row):
        if not isValid(k, width, skip):
          continue

        subframe = frame[j - skip: j + skip + 1, k - skip: k + skip + 1]

        res_channel[i][j][k] = superimpose(subframe, kernel)

  return res_channel[:, skip:-skip, skip:-skip]

In [22]:
def saveAsVideo(frames, file_name, fps):
  frames = frames.astype('uint8')
  writer = imageio.get_writer(file_name, fps=fps, macro_block_size=None)
  for frame in frames:
    writer.append_data(frame)

In [25]:
#driver code
files = os.listdir(path)

frames = np.array(extractFrames(os.path.join(path, files[5]), 30))
video_r_channel, video_g_channel, video_b_channel = preProcessing(frames)

In [26]:
saveAsVideo(video_r_channel, 'video_r.mp4', 5)
saveAsVideo(video_g_channel, 'video_g.mp4', 5)
saveAsVideo(video_b_channel, 'video_b.mp4', 5)